# `ecco` state-space analysis

In this notebook, we see how the state-space of a `rr` model can be computed and analysed symbolically. By _symbolically_ we refer to the use of decision diagrams to represent sets of states in a compact way. More specifically, `ecco` uses [`libDDD` and `libITS`](https://lip6.github.io/ITSTools-web) as its basis for state-space representation and computation. The main benefit of such symbolic methods is that `ecco` can handle very large state-spaces, for models with tens of variables and more, while always maintaining a readable view of the model. However, a consequence is that the individual states are not shown during the analysis. (Another notebook will show how `ecco` can handle explicit state-spaces, that is state-space representations in which individual states are explicitely enumerated.)

## Building and displaying a view

To start with, as usual, we run `ecco` and load a `rr` model:

In [1]:
%run -m ecco termites.rr

Object `model` created by `ecco` can be called as a function, in which case it returns a component view object that allows to explore the models' state space (that is, a component graph). This call requires one argument that is a name for the view (which will be used to save its files). Additional arguments may be provided:
 * `force=True` (default is `False`) to force recomputing all the data associated with the view
 * `compact=False` (default is `True`) to keep the transient states and constraints occurrences in the state space
 
On the first call (or when `force=True`), quite a lot of computation is performed and may take some time:
 * the `rr` model is compiled to an optimised library for faster execution 
 * the symbolic state space is fully computed
 * a basic decomposition is performed extracting various pairwise-disjoint components (each of which may be empty):
   * _hull_ contains all the SCC and the states connecting them
   * _head_ contains all the states leading to _hull_
   * _tail_ contains all the states that are reachable from _hull_
   * _skip_ contains all the states that connect _head_ to _tail_ without going through _hull_
   * _rest_ contains all the other states (for instance, deadlocks reachable from _head_ or _skip_ but not from _hull_)
 * these sets of states are called _components_ and form a _component graphs_ that is saved to the disk into CSV files
 * the content of these tables is available as `.nodes` and `.edges` attributes of the view

In [2]:
v = model("tuto", compact=False)

A view may be drawn, yielding an interactive graph with additional information:
 * default nodes shapes are:
   * circles for SCC
   * squares for components that contain deadlocks
   * rounded squares for all the other components
 * some nodes may be marked with a small badge:
   * a circle for nodes that are SCC hull
   * a triangle for nodes that contain an initial states
 * default color reflects the components size (green for the smaller ones and red for the larger ones)
   
Remember to use `v.draw?` to see the method documentation.

In [3]:
v.draw(fig_height=300)

Information about the components (node) and the edges between them is available by selecting nodes in the graph, or directly through the tables `v.nodes` and `v.edges`.

## Statistical information about components

Method `v.count` allows to count for each component and each variable how many states have this variable on. If no argument is provided to `v.count`, it computes this for every component, otherwise, it expects the list of components for which the information has to be computed:

In [4]:
v.count()

,Ac,Ec,Fg,Rp,Sd,Te,Wd,Wk
0,10,10,8,10,4,4,2,7
1,1,0,0,1,0,0,0,0
2,14,7,6,0,0,8,6,0


This returns a `pandas.DataFrame` whose columns are the variables and whose index (the left-most column withot title) are the components numbers. It is possible to compute a PCA on this table using `v.pca()`. Note also that PCA is one of the layout engines proposed for the graphs of views: it sets the position of nodes according to the result of `v.pca`, considering the first factor as the `x` position and the second factor as the `y` position.

In [15]:
v.pca()

,0,1
0,-2.326425,-1.663681
1,2.993072,-0.754523
2,-0.666646,2.418204


## Splitting components

A component may be split in two according to the valuation of its states variables. For instance below, we split component `2` by telling apart those states in which `Sd` in on from those in which `Sd` is off. The former will become the new component `2` while the latter will go to a new component (that will be numbered `3`).

In [5]:
v.split(0, "Sd")

HTML(value='<p style="line-height:140%"><span style="color:#000088; font-weight:bold;">info</span> detected CT…

In [8]:
v.draw()

When called with no split criterion as second argument, `split` will perform a basic decomposition of the component, just like the state space was initially decomposed (into hull, head, tail, skip, and rest). Moreover, if the split criterion is provided and `normalise=True` also, every component resulting from the split will be further examined for such a basic decomposition.

In general, the split criterion may be an expression using one of the following syntaxes:
 * CTL formulas
 * ARCTL formulas
 * states formulas

Note that the syntax is automatically detected and `ecco` shows which syntax it has detected and used.
 
### CTL formulas

CTL (Computation Tree Logic) is a temporal logic that allows to characterise states with respect to what happens or not in the states that are reachable in the future. A formula can be seen as a statement about a state `s`, that is validated by exploring the states reachable from `s`. When a CTL formula is used to split a component, `ecco` separates the states that validate the formula from those that does not and splits the component accordingly. CTL formulas have to respect the following syntax:
 * atoms are variable names, they may be quoted as in `"AG"` or `'EX'` to avoid conflicts with reserved keywords of CTL, sucha formula is true on every state where the variable is on
 * sub-formulas may be enclosed into parentheses to force operators priority
 * Boolean operators can be used to combine sub-formulas:
   * `~form` (NOT) is a formulat that holds on states where `form` does not
   * `left & right` (AND) holds on states where both `left` and `right` sub-formulas do
   * `left | right` (OR) holds on states where either `left` or `right` sub-formulas does, possibly both
   * `left => right` (IMPLY) holds on states where when `left` holds then `right` has to hold also, this is actually a shorthand for `~left | right`
   * `left <=> right` (IFF) is a shorthand for `(left => right) & (right => left)`
 * modalities allow to express conditions with respect to the future of states: `X` (NEXT), `F` (FUTURE), `G` (GLOBALLY), `U` (UNTIL), and `R` (RELEASE). Each modality must be quantified by either `A` (ALWAYS), or `E` (EXISTS). So a formula may be either:
   * `A path` holds on a state `s` if `path` does on all path starting from `s`
   * `E path` holds on a state `s` if `path` does on one paths starting from `s`
   
   `path` must then be a path formula, that is one formula qualified with a unary modality of two formulas connected by a binary modality:
   * `X form` holds if `form` holds on the next state
   * `F form` holds if `form` holds eventually in the future
   * `G form` holds if `form` holds from now on and forever
   * `left U right` holds if `left` holds until `right` holds forever
   * `left R right` holds if `right` holds until a state where `left` holds is reached, but then `left` or `right` are not required to hold anymore

#### Examples

 * `AX Sd` (_all the next states have soldiers_) selects all the states from which the next state alway has `Sd` on
 * **TODO**

### ARCTL formulas

ARCTL is a variant of CTL where quantifiers apply to a subset of actions. For instance `A{a|b}X Sd` is like `AX Sd` but only considering actions `a` or `b`. These actions are specified in the `rr` model by adding labels to rules or constraints, for instance in our termites model, we could label some rules with a letter indicating the main actor involved into each rule:

```
    [r] Rp+ >> Ec+
    [r] Rp+, Ec+ >> Wk+
    [w] Wk+ >> Wd+, Te+, Fg+, Ec+
    [w] Wk+, Wd+ >> Sd+, Rp+
    [w] Wk+, Te+ >> Wd-
        Wd- >> Wk-, Te-
        Wk- >> Fg-, Sd-
        Wk-, Rp- >> Ec-
    [a] Ac+, Sd- >> Wk-, Rp-

```

In general, labels are indicated into square brackets and are given as a comma separated list of words. For instance `[foo,bar,42]` would label a rule with actions`foo`, `bar`, and `42`. Actions are optional both in the `rr` syntax and the ARCTL syntax. In ARCTL, actions are specified through Boolean expressions constructed using the actions as atoms connected with the Boolean operators `&`, `|`, `=>`, `<=>`, and `~`, as well as parentheses.

#### Examples

 * **TODO**

### States formulas

A state formula allows to select a sets of states based on their features, but, contrary to (AR)CTL, with no reference to the successor states in the execution. Syntax is as follows:
 * the atoms are
   * a variable name as `Wk` which means that we want all the states in which `Wk` is on
   * a rule name as `R3` (or a constraint name as `C1`) which means that we want all the states in which the rule is enabled (ie, may be executed)
 * the operations are
   * `~expr` (NOT) which mean that we want all the states that are not represented by `expr`
   * `left | right` (OR) which means that we want all the states that are in `left`, `right`, or both
   * `left & right` (AND) which means that we want all the states that are both in `left` and `right`
   * `left ^ right` (XOR) which means that we want all the states that are either in `left` or `right` but not in both
   * `(expr)` to group sub-expressions and enforce operators priorities
   
#### Examples

 * `(Rp & ~Wk) | (R2 & ~R1)` represents all the states in which `Rp` is on and `Wk` of, plus all the states from which `R2` but not `R1` can be executed

### Fairness constraints

**TODO**

## Merging components

Components may be also merged using method `merge` whose arguments are the components numbers to be merged. The resulting component will replace the first provided, inheriting its number. So below, `0` and `4` are merged into component `0`.

In [9]:
v.merge(1,3)

In [10]:
v.draw(fig_height=300)

## Workflow

Using splits and merges is the basic way of progressively decomposing a state-space into a graph that makes sense and exhibits interesting properties. Another useful feature is the ability to drop a component: `v.drop(0)` completely remove component `0`, `v.drop(1,3)` removes components `1` and `3`, etc.

Another useful method is `v.save()` that saves the current state of a component graph into a pair of CSV files (one for the nodes and one for the edges).